# Outsoursing order

**Цель:** Провести сверку данных организаций между внутренним файлом АК и внешним файлом ПГС, выявить совпадающие по ОГРН организации и определить количество таких совпадений. Это позволит оценить, сколько организаций из ПГС уже содержатся в базе АК

**Задачи:**
1. Загрузить данные из файлов ПГС и АК 
2. Нормализовать наименования организаций для корректного сравнения
3. Сопоставить записи двух таблиц по значению ОГРН
4. Посчитать количество совпадающих записей по ОГРН
5. Определить, сколько из этих совпадающих записей имеют идентичные или различающиеся наименования организаций

## Загрузка данных и изучение общей информации

In [2]:
#Импортируются библиотеки pandas (для работы с табличными данными)

import pandas as pd

In [3]:
#Задаются настройки вывода, чтобы в консоли выводилось до 200 строк и столбцов, а ширина ячеек могла быть до 1000 символов. 
#Это помогает видеть больше данных при выводе.

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 1000)

In [4]:
#Загружается Excel-файл с информацией об организациях, пропуская первую строку (в файле первая строка - неправильные заголовки)
#Считая следующую строку заголовком столбцов (в файле вторая строка - нужные заголовки)

pgs = pd.read_excel('/Users/anastasiaklockova/Downloads/Выгрузка_организаций_с_ОГРН.xlsx', 
                    skiprows=1, header=0)
pgs.head(2)

,Идентификатор,ОГРН,ИНН,Наименование,ОКАТО,ОКТМО,АДРЕС_в_ГАР
0,6787bf994de6c36845a090ec,0000000000000,NaN,Третий территориальный сектор по охране и воспроизводству объектов животного мира (Череповецкий МР),NaN,NaN,NaN
1,64359d5241fb3800075c2b77,0000000000000,NaN,Вожегодский территориальный отдел – государственное лесничество,NaN,NaN,NaN


In [5]:
#Использую delimiter, потому что файл представлен без предобработки, столбы разделены "\\\"
#Замена на ";" нецелесообразна, ибо в опр.столбце, в самом содержимом (в названии) присутствует ";"
#Заменив, получится так, что разделиться название, а его нужно сохраить целым. 
#В ручную менять везде и проводить предобработку будет затратно по времени

delimiter = r'\\\\\\'

In [6]:
ak = pd.read_csv('/Users/anastasiaklockova/Downloads/esia_organization_test.csv', 
                 delimiter=delimiter,
                engine='python')
ak.head(2)

#параметр engine='python'добавлен, потому что (Warning),сообщает, что из-за сложного разделителябыла автоматически выбрана альтернатива
#движок чтения файла python вместо более быстрого c

,ogrn,inn,full_name,short_name
0,1037821022106,7810239840,"САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ САДОВО-ПАРКОВОЕ ПРЕДПРИЯТИЕ ""ЮЖНОЕ""","СПБ ГУСПП ""ЮЖНОЕ"""
1,1136827000595,6827023407,"МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ДОПОЛНИТЕЛЬНОГО ОБРАЗОВАНИЯ ""СТАНЦИЯ ЮНЫХ ТЕХНИКОВ"" Г.МИЧУРИНСКА ТАМБОВСКОЙ ОБЛАСТИ",МБОУ ДО СЮТ


## Предобработка данных

In [7]:
#Удалим ненужные столбы для сравнения и переименнуем оставшиеся

pgs = pgs.drop(['Идентификатор', 'ОКАТО', 'ОКТМО', 'АДРЕС_в_ГАР'], axis=1, errors='ignore')
pgs = pgs.drop_duplicates(subset=['ОГРН'])

ak = ak.drop(['short_name'], axis=1, errors='ignore') \
       .rename(columns={'ogrn': 'ОГРН', 'inn': 'ИНН', 'full_name':'Наименование'})
ak = ak.drop_duplicates(subset=['ОГРН'])

pgs.head(2)

,ОГРН,ИНН,Наименование
0,0000000000000,NaN,Третий территориальный сектор по охране и воспроизводству объектов животного мира (Череповецкий МР)
34,000000000000000,NaN,-


In [8]:
ak.head(2)

,ОГРН,ИНН,Наименование
0,1037821022106,7810239840,"САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ САДОВО-ПАРКОВОЕ ПРЕДПРИЯТИЕ ""ЮЖНОЕ"""
1,1136827000595,6827023407,"МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ДОПОЛНИТЕЛЬНОГО ОБРАЗОВАНИЯ ""СТАНЦИЯ ЮНЫХ ТЕХНИКОВ"" Г.МИЧУРИНСКА ТАМБОВСКОЙ ОБЛАСТИ"


In [9]:
#Нормализуем наименования, убрав скобки, ковычки в столбце "Наименование"

import re

def normalize_name(name):
    if pd.isnull(name):
        return ''
    # Убираем скобки и содержимое
    name = re.sub(r'\(.*?\)', '', name)
    # Убираем символы кавычек: « », " и '
    name = re.sub(r'[«»"\']', '', name)
    # Приводим к нижнему регистру
    name = name.lower()
    # Убираем лишние пробелы
    name = ' '.join(name.split())
    return name

pgs['norm_name'] = pgs['Наименование'].apply(normalize_name)
ak['norm_name'] = ak['Наименование'].apply(normalize_name)

In [10]:
#Привести наименования к нижнему регистру для унификации регистра и объединить таблицы по ОГРН

merged = pd.merge(pgs, ak, on='ОГРН', suffixes=('_pgs', '_ak'))

matched = merged[merged['norm_name_pgs'] == merged['norm_name_ak']]

# Результат с нужными столбцами
result = matched[['ОГРН', 'ИНН_pgs', 'Наименование_pgs', 'ИНН_ak', 'Наименование_ak']]
result.head(10)

,ОГРН,ИНН_pgs,Наименование_pgs,ИНН_ak,Наименование_ak
3,1000001104555,NaN,ЦАФАП ОДД ГИБДД ГУ МВД России по Красноярскому краю,1001104555,ЦАФАП ОДД ГИБДД ГУ МВД России по Красноярскому краю
4,1000001124099,NaN,ЦАФАП ГИБДД УМВД России по Ивановской области,1001124099,ЦАФАП ГИБДД УМВД России по Ивановской области
5,1000001132067,NaN,Центр АФАП ОДД ГИБДД ГУ МВД России по Кемеровской области,1001132067,Центр АФАП ОДД ГИБДД ГУ МВД России по Кемеровской области
6,1000001146511,NaN,Центр видеофиксации ГИБДД ГУ МВД России по Московской области,1001146511,Центр видеофиксации ГИБДД ГУ МВД России по Московской области
7,1000001153025,NaN,ЦАФАП в ОДД ГИБДД УМВД России по Оренбургской области,1001153025,ЦАФАП в ОДД ГИБДД УМВД России по Оренбургской области
8,1000001161105,NaN,ЦАФАП ГИБДД УМВД России по Рязанской области,1001161105,ЦАФАП ГИБДД УМВД России по Рязанской области
9,1000001163097,NaN,ЦАФАПоДД ГИБДД ГУ МВД России по Саратовской области,1001163097,ЦАФАПоДД ГИБДД ГУ МВД России по Саратовской области
10,1000001185841,NaN,ЦАФАП ОДД ГИБДД МВД по Республике Калмыкия,1001185841,ЦАФАП ОДД ГИБДД МВД по Республике Калмыкия
11,1000001197060,NaN,ЦАФАП ОДД ГИБДД МВД по Чувашской Республике,1001197060,ЦАФАП ОДД ГИБДД МВД по Чувашской Республике
12,1020100003593,105004595.0,"Союз ""Торгово-промышленная палата Республики Адыгея""",0105004595,"СОЮЗ ""ТОРГОВО-ПРОМЫШЛЕННАЯ ПАЛАТА РЕСПУБЛИКИ АДЫГЕЯ"""


In [12]:
# Удаляем дубликаты по столбцам 'ОГРН' и нормализованному названию из matched
matched_unique = matched.drop_duplicates(subset=['ОГРН', 'norm_name_pgs'])

# Количество совпадений по ОГРН и названию
count_name_ogrn = len(matched_unique)

# На основе объединённого датафрейма merged выбираем строки, где нормализованные названия отличаются
mismatch = merged[merged['norm_name_pgs'] != merged['norm_name_ak']]

# Удаляем дубликаты по ОГРН для подсчёта уникальных конфликтов по названию
mismatch_unique = mismatch.drop_duplicates(subset=['ОГРН'])

# Количество совпадений только по ОГРН, но не по названию
count_ogrn_only = len(mismatch_unique)

print('Совпадают по ОГРН и названию:', count_name_ogrn)
print('Совпадают по ОГРН, но названия отличаются:', count_ogrn_only)
total = count_name_ogrn + count_ogrn_only
print('ОГРН есть и в ПГС, и в АК (суммарно):', total)

Совпадают по ОГРН и названию: 15321
Совпадают по ОГРН, но названия отличаются: 6906
ОГРН есть и в ПГС, и в АК (суммарно): 22227


## Итоги сверки


22 227 ОГРН есть и в ПГС, и в АК